In [26]:
# Read GPS data from source file
import pandas as pd

file_path = './GPS_Taxi_third_square.csv'
gps_data = pd.read_csv(file_path)

gps_data.drop(columns='date time', inplace=True)
gps_data['second'] = range(1, len(gps_data) + 1)
gps_data.insert(0, 'second', gps_data.pop('second'))

gps_data.head()

,second,latitude,longitude,accuracy(m),altitude(m),geoid_height(m),speed(m/s),bearing(deg),sat_used,sat_inview
0,1,35.731082,51.531809,12,1354.107,NaN,0.0,NaN,20,24
1,2,35.731083,51.531806,8,1355.046,NaN,0.0,NaN,22,28
2,3,35.731078,51.531807,8,1352.760,NaN,0.0,NaN,24,28
3,4,35.731076,51.531809,8,1350.253,NaN,0.0,NaN,26,29
4,5,35.731081,51.531817,8,1348.108,NaN,0.0,NaN,26,29


In [18]:
# Calculate energy demand
import numpy as np

# Constants from Table 1
mass_vehicle = 11380 + 5100  # total mass (curb + full load) in kg
frontal_area = 8.2  # m^2
drag_coefficient = 0.7  # drag coefficient
rolling_resistance_coefficient = 0.0041  # base coefficient for rolling resistance
air_density = 1.22  # kg/m^3
gravity = 9.81  # m/s^2
transmission_efficiency = 0.96  # transmission efficiency
differential_efficiency = 0.92  # differential efficiency
wheel_radius = 0.5  # approximate value for wheel radius in meters

# Calculate segment-wise forces and energy demand
# Extract speed and elevation from the GPS data
gps_data['speed_kmh'] = gps_data['calculated_speed']  # in km/h
gps_data['speed_ms'] = gps_data['speed_kmh'] / 3.6  # convert to m/s
gps_data['elevation'] = gps_data['altitude(m)']  # using altitude for elevation data

# Calculate elevation difference to determine slope (grade angle)
gps_data['elevation_diff'] = gps_data['elevation'].diff().fillna(0)
gps_data['distance_diff'] = gps_data['distance'].fillna(0)  # distance per segment

# Calculate grade angle (theta) based on elevation difference
gps_data['slope'] = np.arctan(gps_data['elevation_diff'] / gps_data['distance_diff'])

# Apply forces
gps_data['drag_force'] = 0.5 * drag_coefficient * air_density * frontal_area * gps_data['speed_ms']**2
gps_data['rolling_resistance'] = rolling_resistance_coefficient * mass_vehicle * gravity * np.cos(gps_data['slope'])
gps_data['grade_resistance'] = mass_vehicle * gravity * np.sin(gps_data['slope'])

# Calculate tractive force F_x for each segment
gps_data['tractive_force'] = (gps_data['drag_force'] + gps_data['rolling_resistance'] + gps_data['grade_resistance']) / (transmission_efficiency * differential_efficiency)

# Calculate energy demand (work done by tractive force over distance)
gps_data['energy_demand'] = gps_data['tractive_force'] * gps_data['distance_diff']  # in Joules

# Sum total energy demand across all segments
total_energy_demand = gps_data['energy_demand'].sum()  # in Joules
total_energy_demand_kWh = total_energy_demand / (3.6 * 1e6)  # convert to kWh

# Convert to hydrogen fuel requirement
hydrogen_energy_content_kWh_per_kg = 33  # kWh/kg for hydrogen
hydrogen_needed_kg = total_energy_demand_kWh / hydrogen_energy_content_kWh_per_kg

total_energy_demand_kWh, hydrogen_needed_kg


'14:44'